##  Mô tả

Notebook này thực hiện toàn bộ quy trình tiền xử lý dữ liệu trước khi xây dựng mô hình dự đoán churn của khách hàng.  
Các bước bao gồm: đọc và làm sạch dữ liệu, tách loại biến, xử lý giá trị thiếu, mã hoá biến phân loại, chuẩn hoá đặc trưng, tạo thêm feature mới và lưu lại dữ liệu đã xử lý để sử dụng cho quá trình huấn luyện.

## Đọc và chuẩn hóa dữ liệu thô (`BankChurners.csv`)

Đọc file dữ liệu gốc *BankChurners.csv* bằng `numpy.genfromtxt` với kiểu dữ liệu chuỗi (`dtype=str`).

Sau khi đọc dữ liệu, tiến hành chuẩn hóa chuỗi:
- Loại bỏ khoảng trắng dư thừa ở đầu/cuối chuỗi (`np.char.strip`)
- Loại bỏ ký tự ngoặc kép `"` thường xuất hiện do định dạng CSV
- Tách phần header (tên cột) và phần data (dữ liệu)

In [1]:
import numpy as np

# Đọc file
raw = np.genfromtxt(
    "../data/raw/BankChurners.csv",
    delimiter=",",
    dtype=str
)

# 2. Chuẩn hóa
raw = np.char.strip(raw)          # bỏ space
raw = np.char.strip(raw, '"')     # bỏ ký tự "

header = raw[0]      # mảng tên cột
data = raw[1:]       # phần dữ liệu

print("Kích thước dữ liệu:", data.shape)
print("Tên cột:", header)


Kích thước dữ liệu: (10127, 23)
Tên cột: ['CLIENTNUM' 'Attrition_Flag' 'Customer_Age' 'Gender' 'Dependent_count'
 'Education_Level' 'Marital_Status' 'Income_Category' 'Card_Category'
 'Months_on_book' 'Total_Relationship_Count' 'Months_Inactive_12_mon'
 'Contacts_Count_12_mon' 'Credit_Limit' 'Total_Revolving_Bal'
 'Avg_Open_To_Buy' 'Total_Amt_Chng_Q4_Q1' 'Total_Trans_Amt'
 'Total_Trans_Ct' 'Total_Ct_Chng_Q4_Q1' 'Avg_Utilization_Ratio'
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']


## Tạo bảng tra cứu vị trí cột (`col_idx`)

Tạo một từ điển ánh xạ từ tên cột → chỉ số cột giúp việc truy cập và xử lý dữ liệu theo tên cột trở nên dễ dàng và rõ ràng hơn.


In [2]:
col_idx = {name: i for i, name in enumerate(header)}
col_idx

{'CLIENTNUM': 0,
 'Attrition_Flag': 1,
 'Customer_Age': 2,
 'Gender': 3,
 'Dependent_count': 4,
 'Education_Level': 5,
 'Marital_Status': 6,
 'Income_Category': 7,
 'Card_Category': 8,
 'Months_on_book': 9,
 'Total_Relationship_Count': 10,
 'Months_Inactive_12_mon': 11,
 'Contacts_Count_12_mon': 12,
 'Credit_Limit': 13,
 'Total_Revolving_Bal': 14,
 'Avg_Open_To_Buy': 15,
 'Total_Amt_Chng_Q4_Q1': 16,
 'Total_Trans_Amt': 17,
 'Total_Trans_Ct': 18,
 'Total_Ct_Chng_Q4_Q1': 19,
 'Avg_Utilization_Ratio': 20,
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1': 21,
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2': 22}

## Bỏ 2 cột `Naive Bayes`

In [3]:
drop_cols = [
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
]

keep_cols = [c for c in header if c not in drop_cols]

# Lấy chỉ những cột cần thiết
data = data[:, [col_idx[c] for c in keep_cols]]

# Tạo lại col_idx mới
col_idx = {name: i for i, name in enumerate(keep_cols)}

## Xác định các cột Numerical và Categorical

Dữ liệu được chia thành hai nhóm chính:

- Numerical columns: các biến dạng số, dùng cho thống kê mô tả, tiền xử lý (chuẩn hoá), và mô hình học máy.
- Categorical columns: các biến phân loại, thường được xử lý bằng encoding (One-hot, Label Encoding, v.v.).

Việc tách riêng hai nhóm giúp quá trình phân tích và tiền xử lý trở nên mạch lạc và chuẩn hoá hơn.


In [4]:
numerical_cols = [
    "Customer_Age", "Dependent_count", "Months_on_book",
    "Total_Relationship_Count", "Months_Inactive_12_mon",
    "Contacts_Count_12_mon", "Credit_Limit", "Total_Revolving_Bal",
    "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1",
    "Total_Trans_Amt", "Total_Trans_Ct", "Total_Ct_Chng_Q4_Q1",
    "Avg_Utilization_Ratio"
]

categorical_cols = [
    "Gender", "Education_Level", "Marital_Status",
    "Income_Category", "Card_Category"
]

selected_cols = numerical_cols + categorical_cols

print("Số cột numerical:", len(numerical_cols))
print("Số cột categorical:", len(categorical_cols))


Số cột numerical: 14
Số cột categorical: 5


## Xử lý giá trị thiếu (missing values)

Dữ liệu chứa một số ô trống (`""`), thường xuất hiện khi người nhập liệu bỏ qua trường hoặc dữ liệu bị lỗi định dạng.  
Các giá trị rỗng này được thay thế bằng nhãn `"Unknown"` để dễ dàng thống kê và xử lý trong các bước tiếp theo.


In [5]:
# Thay chuỗi rỗng
data[data == ""] = "Unknown"

print("Số giá trị Unknown:", np.sum(data == "Unknown"))


Số giá trị Unknown: 3380


## Tạo biến mục tiêu (target variable)

Biến `Attrition_Flag` là nhãn cho biết khách hàng có rời đi hay không.  
Ta chuyển giá trị `"Attrited Customer"` thành `1` (churn) và các giá trị còn lại thành `0` (non-churn).  
Kết quả được lưu vào biến `y` để phục vụ phân tích và huấn luyện mô hình phân loại.

In [6]:
attr = data[:, col_idx["Attrition_Flag"]]
y = np.where(attr == "Attrited Customer", 1, 0)

print("Tỉ lệ churn:", y.mean() * 100, "%")


Tỉ lệ churn: 16.065962279055988 %


## Hàm mã hoá biến phân loại (Label Encoding)

Hàm `encode_column` dùng để mã hoá một cột dữ liệu phân loại thành các giá trị số nguyên (Label Encoding).
- Lấy danh sách giá trị duy nhất của cột (`np.unique`)
- Tạo ánh xạ từ giá trị chuỗi sang số nguyên
- Áp dụng ánh xạ đó cho toàn bộ cột để tạo vector đã mã hoá

Giá trị trả về

- `encoded`: mảng NumPy chứa các mã số.

- `mapping`: dictionary ánh xạ {giá_trị_gốc: mã_số}.

In [7]:
def encode_column(col):
    uniq = np.unique(col)
    mapping = {v: i for i, v in enumerate(uniq)}
    encoded = np.vectorize(mapping.get)(col)
    return encoded.astype(float), mapping

encoders = {}

## Xây dựng ma trận đặc trưng `X`

Tập dữ liệu đặc trưng được tạo bằng cách duyệt qua danh sách `selected_cols`:

- Với numerical columns, giá trị chuỗi được ép kiểu sang `float` và được chuẩn hoá theo công thức Z-score:
$
X_{\text{norm}} = \frac{X - \text{mean}}{\text{std}}
$

- Với categorical columns, giá trị được mã hoá bằng hàm `encode_column`.

Các cột sau xử lý được ghép lại để tạo thành ma trận `X`.

In [8]:
X_list = []

for col in numerical_cols:
    col_data = data[:, col_idx[col]].astype(float)
    X_list.append(col_data)

# Z-score normalize
num_matrix = np.column_stack(X_list)
mean = num_matrix.mean(axis=0)
std  = num_matrix.std(axis=0) + 1e-9
num_norm = (num_matrix - mean) / std

X_list = [num_norm[:, i] for i in range(num_norm.shape[1])]

# Categorical
for col in categorical_cols:
    encoded, mapping = encode_column(data[:, col_idx[col]])
    encoders[col] = mapping
    X_list.append(encoded)

# Ghép toàn bộ
X = np.column_stack(X_list).astype(float)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (10127, 19)
y shape: (10127,)


## Lưu dữ liệu đã xử lý

Dữ liệu sau tiền xử lý được lưu vào một tệp duy nhất `.npz` để sử dụng cho giai đoạn huấn luyện và đánh giá mô hình. File này bao gồm:

- `X`: ma trận đặc trưng cuối cùng

- `y`: biến mục tiêu

- `mean` và `std`: tham số chuẩn hoá cho các cột `numerical`

- `selected_cols`, `numerical_cols`, `categorical_cols`: danh sách các cột được sử dụng trong mô hình

In [9]:
import numpy as np

np.savez(
    "../data/processed/preprocessed_data.npz",
    X=X,
    y=y,
    mean=mean,
    std=std,
    selected_cols=np.array(selected_cols),
    numerical_cols=np.array(numerical_cols),
    categorical_cols=np.array(categorical_cols)
)

print("Đã lưu preprocessed_data.npz")

Đã lưu preprocessed_data.npz
